# User Based Collaborative Filtering

The User-Based Collaborative Filtering Recommender is a system that suggests items based on the preferences of similar users to the target.

i.e. users who agreed in the past tend to agree again in the future.

We analyse user ratings of courses to identify users with similar 'tastes' and leverage their past opinions to predict what the current user might like.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from coursemate.dataset import Dataset
from coursemate.model import UserBasedCF

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


In [22]:
dataset = Dataset('data/Coursera_courses.csv', 'data/Coursera.csv', 'data/Coursera_reviews.csv')
dataset.set_interaction_counts(3, 50)
dataset.show_dataset_details()

Loading Coursera courses...
Loading Coursera reviews...
Segmenting out students with less than 3 or more than 50 reviews...
30719 students, 468 courses, 174219 reviews
Sparsity: 1.21%
Duplicates: 4.54%


In [23]:
dataset.set_train_test_split_by_user()


Setting the train-test split by user...


In [4]:

# Instantiate the UserBasedCF model
user_cf_model = UserBasedCF()

# Fit the model with the training data
# Ensure that dataset.train_ratings is available and properly set up before this step
user_cf_model.fit(dataset.train_ratings)

In [5]:
df = user_cf_model.predict_all_ratings(dataset.test_ratings)

In [8]:
df

""


In [13]:
# Get recommendations for a particular user
user_id = 'By Kelvin k'
recommendations = user_cf_model.recommend(user_id, k=5)

In [14]:
recommendations

['neural-networks-deep-learning',
 'machine-learning-projects',
 'introduction-tensorflow',
 'python-data',
 'deep-neural-network']

In [24]:
train_Xmatrix, train_ymatrix, df_train_X, df_train_y = dataset.get_train_matrix_split(ratio=0.8)
test_Xmatrix, test_ymatrix, df_test_X, df_test_y = dataset.get_test_matrix_split(ratio=0.5)

Computing the training and test rating matrix...


131100it [00:09, 13556.25it/s]


Computing the test rating matrix split...
Computing the training and test rating matrix...


131100it [00:08, 14993.10it/s]
43119it [00:02, 15752.45it/s]


In [6]:
user_based_cf_model = UserBasedCF()

all_train = pd.concat([df_train_X, df_train_y])

user_based_cf_model.fit(all_train)

In [26]:
df = user_based_cf_model.predict_all_ratings(df_test_X)

100%|██████████| 30719/30719 [18:04<00:00, 28.31it/s]  


In [11]:
df.to_csv('user_based_collab_results_commas.csv', sep=',')

In [35]:
df

,user_id,course_id,predicted_rating
0,By B R s,intro-sql,5.0
1,By B R s,sql-for-data-science,5.0
2,By B R s,understanding-visualization-data,5.0
3,By B R s,google-cloud-product-fundamentals,4.0
4,By B R s,python,4.0
...,...,...,...
106420,By �Sadman S R,initiating-planning,4.0
106421,By �Sadman S R,sql-data-science,3.0
106422,By �Sadman S R,stem-cells,3.0
106423,By �Sadman S R,open-source-tools-for-data-science,2.0


In [36]:
dataset.test_ratings

,reviews,reviewers,date_reviews,rating,course_id
43,Nice course,By A O,2019-07-17,5,python-network-data
44,Nice course,By A O,2019-08-18,5,python-databases
45,Great way to get an introduction into AI and how it can help business and the future.,By A O,2020-02-21,5,ai-for-everyone
191,"to learn how to learn is really awesome program , i really like it so much .",By J,2017-12-16,5,learning-how-to-learn
192,it is really great course for medical students. i just love it . The way of teaching of professor is very simple and very effective. :),By J,2017-12-16,5,physiology
...,...,...,...,...,...
420167,Excellent introductory course.,By 龚子轩,2019-07-28,5,introduction-to-ai
420168,"Patient, detailed, amaizing course!",By 龚子轩,2019-09-28,5,machine-learning-projects
420257,Good!,By 배병선,2019-10-31,4,fundamentals-of-reinforcement-learning
420258,Good Lecture for beginner of deep learning,By 배병선,2020-03-22,4,neural-networks-deep-learning


In [46]:
dataset.test_ratings['reviewers'].unique().shape

df

,user_id,course_id,predicted_rating
0,By B R s,intro-sql,5.0
1,By B R s,sql-for-data-science,5.0
2,By B R s,understanding-visualization-data,5.0
3,By B R s,google-cloud-product-fundamentals,4.0
4,By B R s,python,4.0
...,...,...,...
106420,By �Sadman S R,initiating-planning,4.0
106421,By �Sadman S R,sql-data-science,3.0
106422,By �Sadman S R,stem-cells,3.0
106423,By �Sadman S R,open-source-tools-for-data-science,2.0


In [48]:
df['user_id'].nunique()


# Filter for users in the test set
df2 = df[df['user_id'].isin(dataset.test_ratings['reviewers'].unique())]

df2

,user_id,course_id,predicted_rating


### Evaluation

In [5]:
def calculate_hit_rate(model, test_X, test_y, k=5):
    hit_count = 0
    total = 0

    for user_id in tqdm(dataset.test_students):
        user_history = test_X[test_X['reviewers'] == user_id]['course_id'].values
        actual_next_courses = test_y[test_y['reviewers'] == user_id]['course_id'].values
        
        if len(actual_next_courses) == 0:
            continue

        recommended_courses = model.recommend(user_id, k=k)
        
        print(user_history)
        print(actual_next_courses)
        print(recommended_courses)

        hits = any(course in recommended_courses for course in actual_next_courses)
        hit_count += 1 if hits else 0
        total += 1
        
        print(f"curr {hit_count / total}")
        
        # break

    # Calculate overall hit rate
    return hit_count / total

In [6]:
user_based_cf_model = UserBasedCF()

all_train = pd.concat([df_train_X, df_train_y])

user_based_cf_model.fit(all_train)

In [7]:
calculate_hit_rate(user_based_cf_model, df_test_X, df_test_y, k=20)

  0%|          | 6/7680 [00:00<04:10, 30.65it/s]

['professional-emails-english' 'excel-essentials'
 'managing-human-resources']
['excel-intermediate-1']
[]
curr 0.0
['technical-support-fundamentals' 'os-power-user']
['system-administration-it-infrastructure-services']
[]
curr 0.0
['technical-support-fundamentals' 'os-power-user'
 'system-administration-it-infrastructure-services']
['it-security']
[]
curr 0.0
['what-is-datascience' 'python-for-applied-data-science-ai'
 'sql-data-science']
['data-analysis-with-python' 'python-for-data-visualization']
[]
curr 0.0
['python-network-data' 'introduction-psych' 'psychological-first-aid'
 'negotiation-skills' 'marketing-digital']
['initiating-planning' 'six-sigma-principles' 'project-planning'
 'project-planning']
[]
curr 0.0
['introduction-tensorflow' 'convolutional-neural-networks-tensorflow'
 'deep-neural-network' 'machine-learning-projects' 'nlp-sequence-models'
 'natural-language-processing-tensorflow']
['tensorflow-sequences-time-series-and-prediction' 'python-network-data'
 'python-dat

  0%|          | 15/7680 [00:00<03:32, 36.14it/s]

['gcp-infrastructure-core-services' 'gcp-fundamentals'
 'gcp-infrastructure-foundation']
['gcp-infrastructure-scaling-automation'
 'preparing-cloud-associate-cloud-engineer-exam']
[]
curr 0.0
['python-network-data' 'analytics-excel']
['database-management']
[]
curr 0.0
['neural-networks-deep-learning' 'python-data' 'nlp-sequence-models']
['python-functions-files-dictionaries']
[]
curr 0.0
['research-methods' 'negotiation-skills' 'matlab'
 'data-analysis-with-python' 'python-for-data-visualization'
 'excel-intermediate-1']
['cs-fundamentals-1' 'cs-fundamentals-2' 'technical-support-fundamentals'
 'excel-essentials']
[]
curr 0.0
['uva-darden-foundations-business-strategy' 'data-scientists-tools']
['r-programming']
[]
curr 0.0
['technical-support-fundamentals' 'computer-networking' 'ai-for-everyone']
['system-administration-it-infrastructure-services' 'it-security']
[]
curr 0.0
['neural-networks-deep-learning' 'deep-neural-network'
 'machine-learning-projects']
['nlp-sequence-models']
[]


  0%|          | 24/7680 [00:00<03:22, 37.71it/s]

...

100%|█████████▉| 7677/7680 [03:43<00:00, 19.81it/s]

['machine-learning-projects' 'r-programming'
 'uva-darden-design-thinking-innovation' 'algorithmic-toolbox'
 'linear-algebra-machine-learning']
['learning-how-to-learn' 'seo-fundamentals' 'the-science-of-well-being']
[]
curr 0.0
['technical-support-fundamentals' 'technical-support-fundamentals']
['uva-darden-getting-started-agile']
[]
curr 0.0
['neural-networks-deep-learning' 'machine-learning-projects'
 'nlp-sequence-models']
['introduction-tensorflow' 'convolutional-neural-networks-tensorflow']
[]
curr 0.0
['six-sigma-principles' 'technical-support-fundamentals']
['os-power-user']
[]
curr 0.0
['ds' 'open-source-tools-for-data-science' 'data-science-methodology'
 'sql-data-science' 'data-analysis-with-python']
['python-for-data-visualization' 'machine-learning-with-python'
 'applied-data-science-capstone']
[]
curr 0.0


100%|██████████| 7680/7680 [03:43<00:00, 34.34it/s]

['excel-essentials' 'bcg-uva-darden-digital-transformation']
['excel-intermediate-1']
[]
curr 0.0


0.0

In [9]:
calculate_hit_rate(user_based_cf_model, df_test_X, df_test_y, k=5)

  0%|          | 0/7680 [00:00<?, ?it/s]

KeyError: "None of [Index(['professional-emails-english', 'excel-essentials', 'managing-human-resources'], dtype='object', name='reviewers')] are in the [index]"

In [ ]:
# Work in progress

def precision_at_k(model, test_set, k=5):
    hits = 0
    total = 0
    skipped_users = 0

    for user_id in test_set['reviewers'].unique():
        if user_id not in model.user_similarity_matrix.index:
            print(f"Skipping usr not found in training set: {user_id}")
            skipped_users += 1
            continue

        user_actual = test_set[test_set['reviewers'] == user_id]
        user_actual_top_k = user_actual.nlargest(k, 'rating')['course_id']
        user_predicted_top_k = model.recommend(user_id, k)

        hits += len(set(user_predicted_top_k) & set(user_actual_top_k))
        total += k

    print(f"Processed {len(test_set['reviewers'].unique()) - skipped_users} users, Skipped {skipped_users} users")
    return hits / total if total > 0 else 0

# Test the model
# precision = precision_at_k(user_cf_model, dataset.test_ratings, k=5)
# print(f"Precision@5: {precision}")


Skipping usr not found in training set: By Maggie P
Skipping usr not found in training set: By Ricardo I
Skipping usr not found in training set: By Rithvika L
Skipping usr not found in training set: By VIJAY N


KeyboardInterrupt: 